In [3]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
import nibabel as nib
import SimpleITK as sitk
import ipywidgets as widgets
from os import path

# Prototype

In [2]:
subjectPath = '../../../Data/OpenfMRI/ds000115_R2.0.0/sub-{0:02d}'
savePath = subjectPath + '/func/sub-{0:02d}_task-letter{1:d}backtask_bold-{2:s}.nii'

In [54]:
subject1 = 9
subject2 = 72

backtest = 0

meanFile1 = nib.load(savePath.format(subject1, backtest, 'mean'))
mean1 = meanFile1.get_fdata()
varianceFile1 = nib.load(savePath.format(subject1, backtest, 'variance'))
variance1 = varianceFile1.get_fdata()

meanFile2 = nib.load(savePath.format(subject2, backtest, 'mean'))
mean2 = meanFile2.get_fdata()
varianceFile2 = nib.load(savePath.format(subject2, backtest, 'variance'))
variance2 = varianceFile2.get_fdata()

In [55]:
fixed = sitk.GetImageFromArray(mean1)

moving = sitk.GetImageFromArray(mean2)


R = sitk.ImageRegistrationMethod()
R.SetMetricAsCorrelation()
R.SetOptimizerAsRegularStepGradientDescent(
        learningRate=2.0,
        minStep=1e-4,
        numberOfIterations=500,
        gradientMagnitudeTolerance=1e-8,
    )
R.SetOptimizerScalesFromIndexShift()
tx = sitk.CenteredTransformInitializer(
    fixed, moving, sitk.Similarity3DTransform()
)
R.SetInitialTransform(tx)

R.SetInterpolator(sitk.sitkLinear)

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")


-------
itk::simple::Similarity3DTransform
 Similarity3DTransform (0x3cb4190)
   RTTI typeinfo:   itk::Similarity3DTransform<double>
   Reference Count: 3
   Modified Time: 5019
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1.02945 0.100981 -0.0488756 
     -0.099738 1.03035 0.0280346 
     0.0513641 -0.0231622 1.03401 
   Offset: [-1.94313, -0.952385, -1.12647]
   Center: [18.627, 27.282, 32.257]
   Translation: [-0.216178, -1.07788, 0.295487]
   Inverse: 
     0.959991 -0.0930084 0.0478984 
     0.0941673 0.960829 -0.0215993 
     -0.0455778 0.026143 0.964244 
   Singular: 0
   Versor: [ -0.012379, -0.0242373, -0.0485324, 0.998451 ]
   Scale = 1.03555

Optimizer stop condition: RegularStepGradientDescentOptimizerv4: Step too small after 43 iterations. Current step (6.10352e-05) is less than minimum step (0.0001).
 Iteration: 44
 Metric value: -0.8881959934517533


In [47]:
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(0)
resampler.SetTransform(outTx)
out = resampler.Execute(moving)

In [48]:
registeredMean = sitk.GetArrayFromImage(out)

In [49]:
vmin = np.min([mean1, registeredImage])
vmax = np.max([mean1, registeredImage])

difference = np.abs(mean1 - registeredImage)
emin = difference.min()
emax = difference.max()

@widgets.interact(idx=(0,registeredImage.shape[2]-1,1))
def show_slice(idx=0):
    plt.figure(figsize=(18,10))
    gs = gridspec.GridSpec(2,2)

    plt.subplot(gs[0,0])
    plt.imshow(mean1[:,:,idx], vmin=vmin, vmax=vmax)
    
    plt.subplot(gs[0,1])
    plt.imshow(registeredImage[:,:,idx], vmin=vmin, vmax=vmax)
    plt.colorbar()

    plt.subplot(gs[1,:])
    plt.imshow(difference[:,:,idx], vmin=emin, vmax=emax)
    plt.colorbar()

    plt.show()


interactive(children=(IntSlider(value=0, description='idx', max=35), Output()), _dom_classes=('widget-interact…

In [50]:
fixed2 = sitk.GetImageFromArray(variance1)
moving2 = sitk.GetImageFromArray(variance2)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed2)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(0)
resampler.SetTransform(outTx)
out2 = resampler.Execute(moving2)

In [51]:
registeredVariance = sitk.GetArrayFromImage(out2)

In [62]:
subject1 = 9
subject2 = 72

backtest = 0

meanFile1 = nib.load(savePath.format(subject1, backtest, 'mean'))
mean1 = meanFile1.get_fdata()
varianceFile1 = nib.load(savePath.format(subject1, backtest, 'variance'))
variance1 = varianceFile1.get_fdata()

meanFile2 = nib.load(savePath.format(subject2, backtest, 'mean'))
mean2 = meanFile2.get_fdata()
varianceFile2 = nib.load(savePath.format(subject2, backtest, 'variance'))
variance2 = varianceFile2.get_fdata()

affine = meanFile2.affine
print(affine)
print(affine[:3,:3])

print(np.reshape(outTx.GetMatrix(), (3,3)))
transformationMatrix = np.reshape(outTx.GetMatrix(), (3,3))

print( affine[:3,:3] @ transformationMatrix)
transformedAffine = affine
transformedAffine[:3,:3] = transformationMatrix @ affine[:3,:3]

nibMeanRegistered = nib.Nifti1Image(mean2, transformedAffine)
nib.save(nibMeanRegistered, savePath.format(subject2, backtest, 'mean-registered'))

nibVarianceRegistered = nib.Nifti1Image(variance2, transformedAffine)
nib.save(nibVarianceRegistered, savePath.format(subject2, backtest, 'variance-registered'))

[[-3.90159512e+00  5.22854269e-01 -7.10055470e-01  1.19711945e+02]
 [ 5.25094688e-01  3.96523404e+00  3.45504060e-02 -1.17333000e+02]
 [-7.08400369e-01  5.95111698e-02  3.93632054e+00 -4.18925323e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-3.90159512  0.52285427 -0.71005547]
 [ 0.52509469  3.96523404  0.03455041]
 [-0.70840037  0.05951117  3.93632054]]
[[ 1.02945073  0.10098076 -0.04887555]
 [-0.09973796  1.03035     0.02803458]
 [ 0.05136411 -0.02316216  1.03401159]]
[[-4.10511971  0.16118328 -0.52885497]
 [ 0.14684939  4.13780311  0.12122489]
 [-0.5330132  -0.10139114  4.1064929 ]]


# Bulk

In [21]:
baseSubject = 1

subjectPath = '../../../Data/OpenfMRI/ds000115_R2.0.0/sub-{0:02d}'
savePath = subjectPath + '/func/sub-{0:02d}_task-letter{1:d}backtask_bold-{2:s}.nii'

baseFile = nib.load(savePath.format(baseSubject, 0, 'mean'))
baseAnatomy = baseFile.get_fdata()
fixed = sitk.GetImageFromArray(baseAnatomy)

subjects = np.arange(2, 102 + 1)

for subject in subjects:
    if not path.exists(subjectPath.format(subject)):
        print('skipping subject {:d}'.format(subject))
        continue

    for backtest in range(3):
        subjectFile = nib.load(savePath.format(subject, backtest, 'mean'))
        subjectAnatomy = subjectFile.get_fdata()
        moving = sitk.GetImageFromArray(subjectAnatomy)

        R = sitk.ImageRegistrationMethod()
        R.SetMetricAsCorrelation()
        R.SetOptimizerAsRegularStepGradientDescent(
                learningRate=2.0,
                minStep=1e-4,
                numberOfIterations=500,
                gradientMagnitudeTolerance=1e-8,
            )
        R.SetOptimizerScalesFromIndexShift()
        tx = sitk.CenteredTransformInitializer(
            fixed, moving, sitk.Similarity3DTransform()
        )
        R.SetInitialTransform(tx)

        R.SetInterpolator(sitk.sitkLinear)

        outTx = R.Execute(fixed, moving)
        transformationMatrix = np.reshape(outTx.GetMatrix(), (3,3))
        translation = outTx.GetTranslation()

        meanFile = nib.load(savePath.format(subject, backtest, 'mean'))
        mean = meanFile.get_fdata()
        varianceFile = nib.load(savePath.format(subject, backtest, 'variance'))
        variance = varianceFile.get_fdata()

        transformedAffine = meanFile.affine.copy()
        transformedAffine[:3,:3] = transformationMatrix @ transformedAffine[:3,:3]
        transformedAffine[:3,3] += translation

        nibMeanRegistered = nib.Nifti1Image(mean, transformedAffine)
        nib.save(nibMeanRegistered, savePath.format(subject, backtest, 'mean-registered'))

        transformedAffine = varianceFile.affine.copy()
        transformedAffine[:3,:3] = transformationMatrix @ transformedAffine[:3,:3]
        transformedAffine[:3,3] += translation

        nibVarianceRegistered = nib.Nifti1Image(variance, transformedAffine)
        nib.save(nibVarianceRegistered, savePath.format(subject, backtest, 'variance-registered'))

    print('transformed subject {:d}'.format(subject))

transformed subject 2
transformed subject 3
transformed subject 4
transformed subject 5
transformed subject 6
transformed subject 7
transformed subject 8
transformed subject 9
transformed subject 10
transformed subject 11
transformed subject 12
transformed subject 13
transformed subject 14
transformed subject 15
transformed subject 16
transformed subject 17
transformed subject 18
transformed subject 19
transformed subject 20
transformed subject 21
transformed subject 22
skipping subject 23
transformed subject 24
skipping subject 25
transformed subject 26
transformed subject 27
transformed subject 28
transformed subject 29
transformed subject 30
transformed subject 31
transformed subject 32
transformed subject 33
transformed subject 34
transformed subject 35
transformed subject 36
transformed subject 37
transformed subject 38
transformed subject 39
transformed subject 40
transformed subject 41
transformed subject 42
transformed subject 43
transformed subject 44
transformed subject 45
tr